# Save bedfiles needed for GREAT scan

The design of our last GREAT runs was as follows:
select all G2 and G1 DHSs that meet a particular mean signal threshold (0.5). All of these DHSs become the backghround set. This decision was made to ensure that the background would be less than 1,000,000 DHSs, for that is the upper limit for GREAT. 

The foreground runs we do are as follows  
Every $G2_{i,j}$ for $i,j = 1..16$ excluding cases where $i=j$(240 foreground variants)
and Every $G1_i$  for $i = 1..16$ (16 foreground variants)

The background is the large canvas background every time.

This notebook generates the files, but then you must still run GREAT. This is done in technical notebook 6-18-19 run the GREAT they hate G2 and 6-18-19 run the GREAT they hate G1



In [1]:
import numpy as np
import pandas as pd

In [2]:
ML = pd.read_table('masterlist_DHSs_733samples_WM20180608_all_indexIDs.txt', header=None, names=['chr', 'start', 'end', 'id', 'ML4', 'ML5', 'ML6', 'ML7', 'ML8', 'ML9', 'ML10'], dtype = str)


# trick is that GREAT only uses hg19. We address that here by selecting only hg19 DHSs

In [5]:
hg19ML =  pd.read_table('DHS_Index_hg19.bed', header=None, names=['chr', 'start', 'end', 'id'], dtype=str)

In [6]:
ML = ML.merge(hg19ML, how='left',on='id',suffixes=('','_hg19'))

In [7]:
ML['mean_signal'] = ML.ML4.values.astype(float)/ML.ML5.values.astype(float)

In [8]:
signalcut = ML['mean_signal'] > 0.5

In [9]:
len(signalcut[signalcut])

1485913

### pre-loaded  cuts for G1 and G2

In [10]:
G1cut = np.load('G1cut.npy')

In [11]:
G2cut = np.load('G2cut.npy')

In [12]:
len(G1cut[G1cut*signalcut.values])

653214

In [13]:
len(G2cut[G2cut*signalcut.values])

197514

In [14]:
Mixture = np.load('2018-06-08NC16_NNDSVD_Mixture.npy')
NormedMixture = Mixture/np.sum(Mixture,axis=0)

### still need to calculate every DHSs primary and secondary components

In [15]:
maxcolor = np.argsort(NormedMixture, axis=0)[-1]
maxcolor_vals = np.sort(NormedMixture, axis=0)[-1]
secondmaxcolor = np.argsort(NormedMixture, axis=0)[-2]
secondmaxcolor_vals = np.sort(NormedMixture, axis=0)[-2]

In [16]:
inhg19 = ~pd.isna(ML['start_hg19'])

# Pretty simple from here. Just paste all DHS indices into bed file as long as they have the right components and are in hg19


In [ ]:
for i in range(16):
    
    cut = maxcolor==i
    print(i, len(cut[(G1cut & inhg19 & signalcut).values * cut]))
    ML[(G1cut & inhg19 & signalcut).values * cut][['chr_hg19','start_hg19','end_hg19']].to_csv('60619_C'+str(i+1)+'G1_HQ_gr8_FG.bed', header=False, index=False, sep='\t')
    

In [ ]:
for i in range(16):
    
    cut = maxcolor==i
    for j in range(16):
        if i == j:
            continue
        jcut = secondmaxcolor == j
        
        print(i, j, len(cut[(G2cut & inhg19 & signalcut).values * cut * jcut]))
        ML[(G2cut & inhg19 & signalcut).values * cut * jcut][['chr_hg19','start_hg19','end_hg19']].to_csv('61819_C'+str(i+1)+'C'+str(j+1)+'G2_HQ_gr8_FG.bed', header=False, index=False, sep='\t')

# the background can now be created by concatenating all these fiels together